# [作業目標]
- 使用 Day 12 剛學到的方法, 對較完整的資料生成離散化特徵
- 觀察上述離散化特徵, 對於目標值的預測有沒有幫助

# [作業重點]
- 仿照 Day 12 的語法, 將年齡資料 ('DAYS_BIRTH' 除以 365) 離散化
- 繪製上述的 "離散化標籤" 與目標值 ('TARGET') 的長條圖

In [1]:
# 載入需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 設定 data_path
dir_data = './data/'

### 之前做過的處理

In [6]:
# 讀取資料檔
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape

(307511, 122)

In [3]:
# 將只有兩種值的類別型欄位, 做 Label Encoder, 計算相關係數時讓這些欄位可以被包含在內
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,M,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,F,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,M,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,F,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,M,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
#app_train.columns

In [ ]:
import seaborn as sns # 另一個繪圖-樣式套件
app_train['YEARS_BIRHT'] = app_train['DAYS_BIRTH']/365
age = app_train[['YEARS_BIRHT','TARGET']]
age.head()
age['equal_width_age'] = pd.cut(age['YEARS_BIRHT'],10)
age.head()

age['equal_width_age'].value_counts()
age_group = age.groupby(['equal_width_age']).sum()


sns.barplot(age_group['TARGET'].index,age_group['TARGET'].values)

## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [57]:
income_data = app_train[['CODE_GENDER','AMT_INCOME_TOTAL']]
#income_data.boxplot('AMT_INCOME_TOTAL')

income_data['equal_width_income'] = pd.qcut(income_data['AMT_INCOME_TOTAL'],12)
income_data['equal_width_income'].value_counts()

income_data.head()

income_group = income_data.groupby(['equal_width_income','CODE_GENDER']).mean()
income_group



/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


AMT_INCOME_TOTAL
equal_width_age         CODE_GENDER                  
(25649.999, 76500.0]    F                62751.299571
                        M                63370.112249
                        XNA                       NaN
(76500.0, 90000.0]      F                87753.101637
                        M                88136.214810
                        XNA                       NaN
(90000.0, 112500.0]     F               109260.492724
                        M               109992.264528
                        XNA                       NaN
(112500.0, 117000.0]    F               116835.830275
                        M               116841.657711
                        XNA                       NaN
(117000.0, 135000.0]    F               132479.261395
                        M               133112.155657
                        XNA             135000.000000
(135000.0, 147150.0]    F               142951.562905
                        M               142956.512938
                        XNA                       NaN
(147150.0, 157500.0]    F               156800.397760
                        M               156934.449145
                        XNA             157500.000000
(157500.0, 180000.0]    F               176712.665853
                        M               177789.551533
                        XNA                       NaN
(180000.0, 202500.0]    F               199953.225883
                        M               200439.620428
                        XNA                       NaN
(202500.0, 225000.0]    F               222949.776602
                        M               223588.353105
                        XNA             207000.000000
(225000.0, 292500.0]    F               264346.355919
                        M               265636.697626
                        XNA             247500.000000
(292500.0, 117000000.0] F               409950.477097
                        M               416362.385220
                        XNA                       NaN